In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
pd.options.display.max_columns = None
from pybaseball import batting_stats, batting_stats_range, pitching_stats_range, cache, playerid_lookup, statcast_batter, statcast_pitcher, statcast
import math
import sqlite3
#from fantasy_utils import Fantasy_Projections
import fantasy_utils as fu
import optimize_lineup as ol
import time
cache.enable()
cache.config.cache_type='csv'
cache.config.save()

In [2]:
fp = fu.Fantasy_Projections()
#df = fp.make_projections(fp.yr)

In [3]:
conn = sqlite3.connect('fantasy_data.db')
conn.create_function('sqrt', 1, math.sqrt)
cursor = conn.cursor()

In [4]:
#keepers Google doc
keepers_url = 'https://docs.google.com/spreadsheets/d/1dwDC2uMsfVRYeDECKLI0Mm_QonxkZvTkZTfBgnZo7-Q/edit#gid=1723951361'

In [5]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [ ]:
ids = fp.load_id_map()

In [6]:
owner_dict = {"owner_id":[30, 3, 36, 41, 4, 27, 38, 44, 1, 29, 42, 45],
              "owner":["9 Grand Kids", 'Brewbirds', 'Charmer', 'Dirty Birds', "Harveys Wallbangers", 'Lil Trump & the Ivanabees', 'Lima Time!', 'Roid Ragers', 'Trouble with the Curve', 'Ugly Spuds', 'Wiscompton Wu-Tang', 'Young Guns'],
             }

In [7]:
pd.read_sql("SELECT name FROM sqlite_master", engine)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,roster
8,stats
9,players2023


In [8]:
position_priority = ['posC', 'pos2B', 'pos3B', 'posSS', 'posOF', 'pos1B', 'posMI', 'posCI', 'posDH', 'posSP', 'posRP']
def stitch_positions(row):
    pos_code = row[position_priority+['posP']]>=1
    return str([i.replace('pos','') for i in list(pos_code[pos_code].index)])

### optimized lineups

In [25]:
pd.read_sql("SELECT * FROM vw_player_week_z WHERE year=2024 ORDER BY z desc limit 12 offset 20", conn)

,year,week,cbsid,zR,zRBI,zHR,zSB,zBA,zHit,zW,zSO,zSvHld,zERA,zWHIP,zPitch,z,SvHld
0,2024,9,2133925,NaN,NaN,NaN,NaN,NaN,NaN,3.934065,3.148069,-0.490059,2.565527,3.860059,13.017662,13.017662,0.0
1,2024,1,2918658,NaN,NaN,NaN,NaN,NaN,NaN,7.583434,6.398048,-0.250057,-0.441847,-0.800060,12.489518,12.489518,0.0
2,2024,8,1937347,NaN,NaN,NaN,NaN,NaN,NaN,4.024738,2.843014,-0.538899,2.969474,3.138684,12.437010,12.437010,0.0
3,2024,2,2901324,16.086251,-0.811576,-0.533433,-0.375906,-0.116428,14.248908,-0.404537,-1.075903,-0.440004,-0.000000,-0.000000,-1.920444,12.328464,0.0
4,2024,6,3126008,NaN,NaN,NaN,NaN,NaN,NaN,3.754746,3.432114,-0.489049,2.459176,2.840005,11.996992,11.996992,0.0
5,2024,10,3117472,1.294603,3.850068,4.127032,0.422262,2.198871,11.892837,NaN,NaN,NaN,NaN,NaN,NaN,11.892837,NaN
6,2024,12,2071264,1.420668,3.654303,2.432123,0.944246,3.109595,11.560936,NaN,NaN,NaN,NaN,NaN,NaN,11.560936,NaN
7,2024,10,2044495,1.626697,3.850068,4.127032,0.422262,1.460985,11.487045,NaN,NaN,NaN,NaN,NaN,NaN,11.487045,NaN
8,2024,11,2071264,2.282265,2.222815,3.694945,1.853947,1.396081,11.450052,NaN,NaN,NaN,NaN,NaN,NaN,11.450052,NaN
9,2024,11,26911736,NaN,NaN,NaN,NaN,NaN,NaN,3.911207,2.821215,-0.516874,2.716950,2.344726,11.277225,11.277225,0.0


In [9]:
weekly_lineup_rating = {}

In [20]:
a[a['all_pos'].isna()].sort_values(['Player', 'week'])

,year,week,cbsid,Player,Owner,pos,z,all_pos
152,2024,5,2910604,Brandon Lowe,Young Guns,2B,-1.861076,None
168,2024,5,2250592,Braxton Garrett,Young Guns,RP,-9.000000,None
146,2024,5,3152047,Brenton Doyle,Young Guns,OF,-0.111184,None
140,2024,5,2942981,Brice Turang,Young Guns,SS,2.529627,None
163,2024,5,1390876,Charlie Morton,Young Guns,SP,2.067716,None
157,2024,5,2114418,Chris Bassitt,Young Guns,SP,3.715481,None
165,2024,5,2044540,Clay Holmes,Young Guns,RP,1.724100,None
145,2024,5,26719565,Colton Cowser,Young Guns,OF,1.301113,None
162,2024,5,2910637,Cristian Javier,Young Guns,SP,-9.000000,None
397,2024,12,2910637,Cristian Javier,Young Guns,SP,-2.207779,None


In [17]:
tm = 'Young Guns'
a = pd.read_sql(f"SELECT r.year, r.week, r.cbsid, COALESCE(p.CBSNAME,'unknown') Player, COALESCE(o.owner,'FA') Owner, \
    r.pos, COALESCE(z.z,-9) z, all_pos \
    FROM roster r \
    LEFT JOIN players p On (r.cbsid=p.cbsid) \
    LEFT JOIN owners o On (r.owner_id=o.owner_id) \
    LEFT JOIN vw_player_week_z z On (z.cbsid=r.cbsid AND z.year=r.year AND z.week=r.week) \
    LEFT JOIN eligibility e On (e.cbsid=r.cbsid AND e.year=r.year AND e.week=r.week) \
    WHERE o.owner='{tm}' and r.year=2024", conn)
a['type'] = a['all_pos'].apply(lambda x: 'h' if 'DH' in x else 'p')

TypeError: argument of type 'NoneType' is not iterable

In [12]:
pd.pivot_table(a, columns='week', aggfunc='count', values='cbsid')

week,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0
cbsid,49,54,53,57,55,54,56,58,60,60,61,59,59,30,30,25,28,28,27,26,27,27,24,30,29,27,25


In [294]:
b = pd.read_sql(f"SELECT COALESCE(p.cbsname,'not known') Player, COALESCE(o.owner, 'FA') Owner, \
    r.*, COALESCE(z.z,-9) z, e.all_pos \
    FROM roster r \
    LEFT JOIN players p On (r.cbsid=p.cbsid) \
    LEFT JOIN owners o On (r.owner_id=o.owner_id) \
    LEFT JOIN vw_player_week_z z On (z.cbsid=r.cbsid AND z.year=r.year AND z.week=r.week) \
    LEFT JOIN eligibility e On (e.cbsid=r.cbsid AND e.year=r.year AND e.week=r.week) \
    WHERE o.owner='{tm}'", conn)
b['type'] = b['all_pos'].apply(lambda x: 'h' if 'DH' in x else 'p')
b

,Player,Owner,cbsid,owner_id,year,week,pos,z,all_pos,type
0,David Robertson,Young Guns,1622557,45,2023,1,RP,2.521829,['P'],p
1,David Robertson,Young Guns,1622557,45,2023,2,RP,0.842537,['P'],p
2,David Robertson,Young Guns,1622557,45,2023,3,RP,5.517404,['P'],p
3,David Robertson,Young Guns,1622557,45,2023,4,RP,-1.526831,['P'],p
4,David Robertson,Young Guns,1622557,45,2023,5,RP,1.345033,['P'],p
...,...,...,...,...,...,...,...,...,...,...
886,Jackson Chourio,Young Guns,26643926,45,2023,23,B,-9.000000,['P'],p
887,Jackson Chourio,Young Guns,26643926,45,2023,24,B,0.000000,['P'],p
888,Jackson Chourio,Young Guns,26643926,45,2023,25,B,0.000000,"['OF', 'DH']",h
889,Jackson Chourio,Young Guns,26643926,45,2023,26,B,-9.000000,"['OF', 'DH']",h


In [295]:
pd.pivot_table(b, columns='week', aggfunc='count', values='cbsid')

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
cbsid,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33


In [273]:
#cursor.execute("INSERT INTO roster VALUES (26615381, 4, 2023, 27, 'DH')")
#cursor.execute("INSERT INTO eligibility VALUES (26615381, 2023, 26, \"['OF', 'DH']\", 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0)")
#cursor.execute("DELETE FROM eligibility WHERE cbsid=26615381 and week=26")
#cursor.execute("UPDATE eligibility SET all_pos=\"['C', 'OF', 'DH']\", posC=1 WHERE cbsid=26719539")

In [297]:
weekly_lineup_rating[tm] = []

for wk in range(1,28):
    p_score = b[(b['pos']!='B') & (b['week']==wk)].z.sum()
    #assert a[a['pos']!='B'].shape[0] == 23
    #assert a.shape[0] == 33

    opt = ol.Optimized_Lineups(tm, b[b['week']==wk])
    opt._make_hitter_combos()
    opt._make_pitcher_combos()
    opt_score = opt.hitter_optimized_z + opt.pitcher_optimized_z
    #print(opt_score)
    print(f"Week {wk}: {round(p_score/opt_score,2)}")
    weekly_lineup_rating[tm].append({'week':wk, 'score':p_score, 'optimal':opt_score, 'rating':round(p_score/opt_score,2)})

Week 1: 0.4
Week 2: 0.82
Week 3: 0.65
Week 4: 0.57
Week 5: 0.43
Week 6: 0.89
Week 7: 0.35
Week 8: 0.3
Week 9: 0.69
Week 10: 0.67
Week 11: 0.27
Week 12: 0.57
Week 13: 0.64
Week 14: 0.8
Week 15: 0.56
Week 16: -0.03
Week 17: 0.27
Week 18: -1.38
Week 19: 0.28
Week 20: -4.92
Week 21: -0.76
Week 22: 140.2
Week 23: 0.4
Week 24: 0.83
Week 25: 0.71
Week 26: 0.68
Week 27: 0.58


In [141]:
#cursor.execute("UPDATE eligibility SET posC=1 WHERE cbsid=2164748")
#cursor.execute('UPDATE eligibility SET all_pos = "[\'C\', \'DH\']" WHERE cbsid=2164748')
#cursor.execute('DELETE FROM roster WHERE cbsid=29075539 AND pos="DH" AND week>=26')
#conn.commit()

In [191]:
max_z = -999
max_z_idx = -1
for num in range(len(position_combos)):
    list_z = sum([opt.h_dict[name][opt.optimize_col] for name in position_combos[num] if type(name)!=tuple]+[opt.h_dict[item][opt.optimize_col] for sublist in position_combos[num] for item in sublist if type(sublist)==tuple])
    if list_z > max_z and len(set(list(position_combos[num][0:-1])+list(position_combos[num][-1]))) == 12:
            max_z = list_z
            max_z_idx = num

In [303]:
weekly_lineup_rating[tm]

[{'week': 1,
  'score': 10.295078571414976,
  'optimal': 25.61138003071205,
  'rating': 0.4},
 {'week': 2,
  'score': 30.747394113944075,
  'optimal': 37.294324661223314,
  'rating': 0.82},
 {'week': 3,
  'score': 16.037957138961687,
  'optimal': 24.751196502144868,
  'rating': 0.65},
 {'week': 4,
  'score': 18.306801908566108,
  'optimal': 32.121358205420016,
  'rating': 0.57},
 {'week': 5,
  'score': 12.418213427257403,
  'optimal': 28.606460389818324,
  'rating': 0.43},
 {'week': 6,
  'score': 12.697581095046312,
  'optimal': 14.222494623035985,
  'rating': 0.89},
 {'week': 7,
  'score': 13.110904015633588,
  'optimal': 37.351343964226665,
  'rating': 0.35},
 {'week': 8,
  'score': 10.334645781922918,
  'optimal': 34.94276625161318,
  'rating': 0.3},
 {'week': 9,
  'score': 29.564498822665925,
  'optimal': 43.042848443366125,
  'rating': 0.69},
 {'week': 10,
  'score': 31.52051995482274,
  'optimal': 47.035587192109844,
  'rating': 0.67},
 {'week': 11,
  'score': 11.477039320930164,

In [307]:
b[b['week']==22].sort_values(['pos','z'])

,Player,Owner,cbsid,owner_id,year,week,pos,z,all_pos,type
489,LaMonte Wade,Young Guns,2211805,45,2023,22,1B,-1.183742,"['OF', '1B', 'CI', 'DH']",h
127,Jon Berti,Young Guns,1957205,45,2023,22,2B,-3.723497,"['2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH']",h
451,Matt Chapman,Young Guns,2135256,45,2023,22,3B,-2.229113,"['3B', 'CI', 'DH']",h
66,Nathan Eovaldi,Young Guns,1803864,45,2023,22,B,-9.000000,['P'],p
397,Trey Mancini,Young Guns,2119352,45,2023,22,B,-9.000000,"['OF', '1B', 'CI', 'DH']",h
810,Alek Manoah,Young Guns,3117538,45,2023,22,B,-9.000000,['P'],p
885,Jackson Chourio,Young Guns,26643926,45,2023,22,B,-9.000000,['P'],p
756,Tommy Edman,Young Guns,2914724,45,2023,22,B,-4.305025,"['2B', 'SS', 'OF', 'MI', 'DH']",h
292,Clay Holmes,Young Guns,2044540,45,2023,22,B,-3.535467,['P'],p
516,Luis Urias,Young Guns,2227185,45,2023,22,B,-1.994650,"['2B', '3B', 'MI', 'CI', 'DH']",h


In [300]:
lt = pd.DataFrame(weekly_lineup_rating['Lima Time!'])
ch = pd.DataFrame(weekly_lineup_rating['Charmer'])
bb = pd.DataFrame(weekly_lineup_rating['Brewbirds'])
hw = pd.DataFrame(weekly_lineup_rating['Harveys Wallbangers'])
us = pd.DataFrame(weekly_lineup_rating['Ugly Spuds'])

tc = pd.DataFrame(weekly_lineup_rating['Trouble with the Curve'])
db = pd.DataFrame(weekly_lineup_rating['Dirty Birds'])
rr = pd.DataFrame(weekly_lineup_rating['Roid Ragers'])
gk = pd.DataFrame(weekly_lineup_rating['9 Grand Kids'])
lti = pd.DataFrame(weekly_lineup_rating['Lil Trump & the Ivanabees'])

ww = pd.DataFrame(weekly_lineup_rating['Wiscompton Wu-Tang'])
yg = pd.DataFrame(weekly_lineup_rating['Young Guns'])

In [302]:
lt.rating.mean(), ch.rating.mean(), bb.rating.mean(), hw.rating.mean(), us.rating.mean(), tc.rating.mean(), db.rating.mean(), rr.rating.mean(), gk.rating.mean(), lti.rating.mean(), ww.rating.mean(), yg.rating.mean()

(0.5896296296296296,
 0.6955555555555557,
 0.5655555555555557,
 0.7870370370370371,
 0.6100000000000001,
 0.5577777777777777,
 0.5703703703703703,
 5.630000000000001,
 0.4914814814814814,
 0.46629629629629626,
 0.6066666666666666,
 5.387777777777779)

In [147]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        name='Lima Time!', x=lt.score, y=lt.optimal, mode='markers', text=lt.week
    )
)
fig.add_trace(
    go.Scatter(
        name='Charmer', x=ch.score, y=ch.optimal, mode='markers', text=ch.week
))
fig.add_trace(
    go.Scatter(
        name='Brewbirds', x=bb.score, y=bb.optimal, mode='markers', text=bb.week
))
fig.add_trace(
    go.Scatter(
        name='Harveys Wallbangers', x=bb.score, y=bb.optimal, mode='markers', text=bb.week
))
fig.update_layout(xaxis_title='player score', yaxis_title='optimal score',
                 yaxis_range=[0,100], xaxis_range=[0,100])
fig.show()